In [ ]:
import cv2 as cv
import numpy as np

#constants
LANE_WIDTH = 340

In [ ]:
def PerspectiveTransform(src,dst):
    mat = cv2.getPerspectiveTransform(src,dst)
    mat_inv = cv2.getPerspectiveTransform(dst,src)
    return mat,mat_inv


def warpPerspective(img, mat, size):
    return cv2.warpPerspective(img, mat,size, cv2.INTER_LINEAR)

#function for converting the image to hls mode then return s as it is the one we are intrested in studying
def trans2hls(image):
    img_hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    (h,l,s) = cv2.split(img_hls)
    return s

In [ ]:
def detectLane(frame, src_pts, dst_pts, size, kernel_op, kernel_cl):
    
    sat = trans2hls(frame)
    
    canny = cv2.Canny(sat,50,150)
    
    #pre-processing the edge extracted image
    
    #performing opening operation to remove extra noise
    
    open_canny = cv2.morphologyEx(canny, cv2.MORPH_OPEN,kernel_op )
    
    
    #performing closing operation to thicken lane lines
    
    close_canny = cv2.morphologyEx(open_canny, cv2.MORPH_CLOSE,kernel_cl)
    
    #performing perspective transform
    mat, mat_inv = PerspectiveTransform(src_pts,dst_pts)
    bird_view_gray = warpPerspective(close_canny,mat,size)
    bird_view = warpPerspective(frame,mat,size)
    #retrans = warpPerspective(bird_view_gray, mat_inv, size)
    
    #detecting lane lines
    lines = cv2.HoughLinesP(bird_view_gray,1, np.pi/180,50,minLineLength=400, maxLineGap=60 )
    
    #line[0] returns x,y reversed where x is horizontal axis and y is the vertical axis
    i=0
    for line in lines:
        x1,y1,x2,y2 = line[0]

        if(x1>350): #in case the second lane line was on the right side
            cv2.line(bird_view,(x1,y1),(x2,y2),red,2)
            if(x1+LANE_WIDTH < 900):
                cv2.line(bird_view,(x1+LANE_WIDTH,y1),(x2+LANE_WIDTH,y2),red,2)
                x3 = x1+LANE_WIDTH
                x4 = x2+LANE_WIDTH
            if(x1-LANE_WIDTH > 300): #in case the second lane line was on the left
                left = 1
                cv2.line(bird_view,(x1-LANE_WIDTH,y1),(x2-LANE_WIDTH,y2),red,2)
                x3 = x1-LANE_WIDTH
                x4 = x2-LANE_WIDTH

            #340 is fixed which represents the lane width
            if(i<=4):
                #getting the boundary of lines to draw the contour(boundary)
    ###### momken a7sen fe deh eny a3mlha b minimum w maximum x1 w x2
                bound_top_left= (x1,y1)
                bound_top_right= (x3,y1)
            
            if(y2 > bound_bottom_left[1]):
            
                bound_bottom_left = (x2,y2)
                if(left == 1):
                    bound_bottom_right = (x2-LANE_WIDTH,y2)
                else:
                    bound_bottom_right = (x2+LANE_WIDTH,y2)








        i=i+1
        
    #drawing the top and bottom boundaries
    cv2.line(bird_view,bound_bottom_left,bound_bottom_right,red,5)
    cv2.line(bird_view,bound_top_left,bound_top_right,red,5)
    
    # drawing the green region representing the lane
    contours = np.array([bound_top_left,bound_top_right,bound_bottom_right, bound_bottom_left])
    cv2.fillPoly(bird_view,pts = [contours],color = green)

    #transforming the view back from bird eye view
    re_gen = warpPerspective(bird_view, mat_inv, size)

    #combning modified frame with original frame
    image = np.zeros_like(frame)
    cv2.addWeighted(frame, 0.5, re_gen, 0.5,0, image)
    
    return image